__Part 1__

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.2)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

D:\PythonProjects\DataScience\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - accuracy: 0.6976 - loss: 0.8288 - val_accuracy: 0.8458 - val_loss: 0.4164
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.8601 - loss: 0.3881 - val_accuracy: 0.8735 - val_loss: 0.3477
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.8833 - loss: 0.3191 - val_accuracy: 0.8872 - val_loss: 0.3102
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.8985 - loss: 0.2784 - val_accuracy: 0.8944 - val_loss: 0.2892
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9082 - loss: 0.2500 - val_accuracy: 0.8952 - val_loss: 0.2885
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9163 - loss: 0.2284 - val_accuracy: 0.9068 - val_loss: 0.2581
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9223 - loss: 0.2113 - val_accuracy: 0.9065 - val_loss: 0.2610
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9293 - loss: 0.1933 - 

__Part 2__

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

train_images = np.stack([train_images] * 3, axis=-1)
test_images = np.stack([test_images] * 3, axis=-1)
train_images = np.array([np.array(tf.image.resize(img, (32, 32))) for img in train_images])
test_images = np.array([np.array(tf.image.resize(img, (32, 32))) for img in test_images])

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
conv_base.trainable = False

model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_images, train_labels,
                    epochs=10,
                    batch_size=64,
                    validation_data=(val_images, val_labels))

conv_base.trainable = True
for layer in conv_base.layers[:15]:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_fine = model.fit(train_images, train_labels,
                         epochs=30,
                         batch_size=64,
                         validation_data=(val_images, val_labels),
                         callbacks=[lr_scheduler, early_stopping])

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy after fine-tuning: {test_acc:.4f}")

Epoch 1/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 330s 435ms/step - accuracy: 0.6895 - loss: 0.9004 - val_accuracy: 0.8287 - val_loss: 0.4720
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 302s 402ms/step - accuracy: 0.8249 - loss: 0.4889 - val_accuracy: 0.8463 - val_loss: 0.4215
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 262s 350ms/step - accuracy: 0.8363 - loss: 0.4544 - val_accuracy: 0.8528 - val_loss: 0.4009
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 234s 312ms/step - accuracy: 0.8460 - loss: 0.4248 - val_accuracy: 0.8586 - val_loss: 0.3891
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 234s 313ms/step - accuracy: 0.8478 - loss: 0.4143 - val_accuracy: 0.8557 - val_loss: 0.3842
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 235s 313ms/step - accuracy: 0.8588 - loss: 0.3878 - val_accuracy: 0.8620 - val_loss: 0.3738
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 221s 295ms/step - accuracy: 0.8563 - loss: 0.3876 - val_accuracy: 0.8642 - val_loss: 0.3672
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 212s 282ms/step - accuracy: 0.8657 -

__MLP:__ Achieved 88.85% accuracy, which is the lowest among the three models. While decent, it shows that MLPs are less effective for image classification tasks compared to CNNs due to their lack of spatial feature extraction capabilities.<br>
__CNN:__ Achieved the highest accuracy at 91.08%. This indicates that a custom CNN designed specifically for the Fashion MNIST dataset can perform very well by effectively capturing spatial hierarchies and patterns in the image data.<br>
__CNN based on VGG16:__ Achieved 90.34% accuracy, slightly lower than the standard CNN but still significantly higher than the MLP. This demonstrates the power of transfer learning and leveraging pre-trained models, even though it didn't outperform the custom CNN in this case.